In [1]:
##Robs summariser of RC8 regoinal Contributor tables
import pandas as pd
import numpy as np
import os

#mainPath = 'P:\projects\RC9_ResultsSets\RegContribTables'
# mainPath = r'P:\projects\RC10_ResultsSets\SummariesWithProcessAusGov'
# reportCardString = 'RC2019'
#mainPath = r'P:\projects\RC8_ResultsSets\SummariesWithProcess'
#reportCardString = 'RC2016'

mainPath = r'P:\projects\RC8_ResultsSets\SummariesWithProcessRC10regions'
reportCardString = 'RC2016'

#outPath = r'P:\projects\RC10_ResultsSets'
#summariesOutFolder = 'SummariesWithProcessAusGov'
# baseFolderEnd = 'BASE_RC10'
# changeFolderEnd = 'CHANGE_RC10'
# predevFolderEnd = 'PREDEV_RC10'
processFileName = 'AnthroRegByLanduseProcess.csv'
outputCSVEnd = 'ProcessExportContribution.csv'

fieldName = 'Export_KG'

#pathToContstitsToGrpFile = r'\\athenasmb\pdreef\RC10_RC2019\RC10_RC2019_ResultsSets_PointOfTruth\constituentsToGroups.csv'

##Must be utilised in regContributors that do NOT include nesting structures
#alternativeRegionsCSV = {'FI':r'\\athenasmb\pdreef\RC10_RC2019\RC10_RC2019_ResultsSets_PointOfTruth\FI_ReportingRegions_Details.csv',
#                        'BU':r'\\athenasmb\pdreef\RC10_RC2019\RC10_RC2019_ResultsSets_PointOfTruth\BU_ReportingRegions_Details.csv'}

#altRegLinker = 'Catchmt'
#standardRegName = 'RepReg'
#altRegColName = 'AUSGMCAS'
#origLinker = 'ModelElement'

annLoadToStream = 'AnnLoadToStreamKG'
annLoadToExport = 'AnnLoadToExportKG'

#regionIDs = ['BU', 'BM', 'CY', 'FI', 'MW', 'WT']
regionIDs = ['BU']
scenarios = ['Base','PreDev','Change']

constitsList = ['Sediment - Fine', 'N_DIN']

#scenarioToFolderDict = {baselineScenarioName:baseFolderEnd, changeScenarioName:changeFolderEnd, predevScenarioName:predevFolderEnd}


In [2]:
# if not os.path.exists(outPath):
#     os.makedirs(outPath)
#     print("Made dir: " + outPath)

In [3]:
# if not os.path.exists(outPath + '\\' + summariesOutFolder):
#     os.makedirs(outPath + '\\' + summariesOutFolder)
#     print("Made dir: " + outPath + '\\' + summariesOutFolder)

In [4]:
def getPathInfo(regionIDString):
    fileIn = mainPath + '\\' + regionIDString + '_' + reportCardString + '_' + processFileName
    fileOut = mainPath + '\\' + regionIDString + '_' + reportCardString + '_' + outputCSVEnd
    return fileIn, fileOut


In [5]:
def produceSummaryFromProcessFile(regionIDString):
    
    theFileIn, theFileOut = getPathInfo(regionIDString)
    
    print("Processing " + theFileIn)
    
    rawcontrib = pd.read_csv(theFileIn)
    
    filtered = rawcontrib.loc[rawcontrib['Constituent'].isin(constitsList)]
    
    groupedData = filtered.groupby(['Source Region','Constituent','Process']).agg({'Base_Export_KG':'sum','PreDev_Export_KG':'sum','Change_Export_KG':'sum'}).reset_index()

    groupedTotals = filtered.groupby(['Source Region','Constituent']).agg({'Base_Export_KG':'sum','PreDev_Export_KG':'sum','Change_Export_KG':'sum'}).reset_index()
    
    groupedTotals = groupedTotals.rename(columns={'Base_Export_KG':'Base_Total_KG',
                                                  'PreDev_Export_KG':'PreDev_Total_KG','Change_Export_KG':'Change_Total_KG'})
    
    mergeddf = pd.merge(groupedData, groupedTotals, how='left'
                        , left_on=['Source Region','Constituent'], right_on=['Source Region','Constituent']).reset_index()
    
    
    mergeddf['BaseContribution'] = mergeddf['Base_Export_KG'] / mergeddf['Base_Total_KG'] * 100
    mergeddf['PreDevContribution'] = mergeddf['PreDev_Export_KG'] / mergeddf['PreDev_Total_KG'] * 100
    mergeddf['ChangeContribution'] = mergeddf['Change_Export_KG'] / mergeddf['Change_Total_KG'] * 100
    
#     #rawcontrib['SCENARIO'] = scenarioName
#     rawcontrib[annLoadToStream] = rawcontrib['LoadToStream (kg)'].div(rawcontrib['Num_Days']).mul(365.25)
#     rawcontrib[annLoadToExport] = rawcontrib['LoadToRegExport (kg)'].div(rawcontrib['Num_Days']).mul(365.25)
#     rawcontrib['AreaHA'] = rawcontrib['AreaM2'].div(10000)
    
#     if regionIDString in alternativeRegionsCSV:
#         print("Reading alternative regionalistaion for " + regionIDString)
#         altDetsDF = pd.read_csv(alternativeRegionsCSV[regionIDString])
#         combinedDF = pd.merge(rawcontrib, altDetsDF, how='left', left_on=[origLinker], right_on = [altRegLinker])
#         #Transfer vals
#         combinedDF['Rep_Region'] = combinedDF[altRegColName]
#         #Drop cols
#         #where 1 is the axis number (0 for rows and 1 for columns.)
#         combinedDF = combinedDF.drop(altRegColName, 1)
#         combinedDF = combinedDF.drop(standardRegName, 1)
#         combinedDF = combinedDF.drop(altRegLinker, 1)
#         rawcontrib = combinedDF
    
#     ### Can keep reporting region in the area table, as this is our Non-Nested notebook
#     catchFuArea = pd.DataFrame(rawcontrib.groupby(['Rep_Region','ModelElement','FU']).agg({'AreaHA':'first'})).reset_index()
#     regFuArea = pd.DataFrame(catchFuArea.groupby(['Rep_Region','FU']).agg({'AreaHA':'sum'})).reset_index()
    
#     #regLuseSummary = pd.DataFrame(rawcontrib[rawcontrib['Constituent'].isin(selectedConstituents)].groupby(['Rep_Region','Constituent','FU','Process']).agg({'AnnLoadToStreamKG':'sum','AnnLoadToExportKG':'sum'})).reset_index()
#     regLuseSummary = pd.DataFrame(rawcontrib.groupby(['SCENARIO','Rep_Region','Constituent','FU','Process']).agg({annLoadToStream:'sum',annLoadToExport:'sum'})).reset_index()
    
#     ### Join/merge with Regional Areas
#     regLuseSumPlusArea = pd.merge(regLuseSummary, regFuArea, how='left', left_on=['Rep_Region','FU'], right_on = ['Rep_Region','FU'])
    
    mergeddf.to_csv(theFileOut, index=False)
    
    print("Saved " + theFileOut)
    
    return

    

In [7]:
#List out the processes

for theReg in regionIDs:
    print("Doing: " + theReg)
    produceSummaryFromProcessFile(theReg)

print("Finished first summary")

Doing: BU
Processing P:\projects\RC8_ResultsSets\SummariesWithProcessRC10regions\BU_RC2016_AnthroRegByLanduseProcess.csv
Saved P:\projects\RC8_ResultsSets\SummariesWithProcessRC10regions\BU_RC2016_ProcessExportContribution.csv
Finished first summary
